# calculo Valor garantia Agrupado Final

In [2]:
##CARGUE DE LIBRERIAS UTILES Y NECESARIAS
import pyodbc # Importando la librería pyodbc para conectarse a bases de datos ODBC
import pandas as pd # Importando la librería Pandas para trabajar con DataFrames
import numpy as np   # Importando la librería Numpy para operaciones matemáticas y estadísticas
import glob # Importando la librería Glob para obtener listas de archivos basadas en patrones
import re # Importando la librería re para trabajar con expresiones regulares
from alive_progress import alive_bar # Importando la barra de progreso alive_progress para mostrar  barras de progreso animadas en la consola
import datetime # Importando la librería datetime para trabajar con fechas y horas
import os # Importando la librería os para interactuar con el sistema operativo (paths, environment variables, etc.)
import timeit
import time
from datetime import datetime, timedelta, date
import calendar
pd.set_option('display.max_columns', None)

In [3]:
inicio = time.time()
time.sleep(1)

In [4]:
# Funciones

def first_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 1)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 1)
    
def last_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 31)
    elif d.month == 3:
        if calendar.isleap(d.year) == True:
            return d.replace(month =d.month -1, year = d.year, day= 29)
        else:
            return d.replace(month =d.month -1, year = d.year, day= 28)
    elif (d.month == 5) | (d.month == 7) |(d.month == 10) | (d.month == 12):
        return d.replace(month =d.month -1, year = d.year, day= 30)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 31)


**Se declaran la fecha correpondiente a los dias primero y ultimo del mes inmediatamente anterior**

In [5]:
# hoy = date.today()
# primero = first_day(hoy).strftime('%Y%m%d')
# ultimo = last_day(hoy).strftime('%Y%m%d')
# ayer =datetime.today()+timedelta(days=-1)
fecha_registro = 202501
ult_dia_mes = 20240131
primero = 20250228
ultimo = 20250228
ayer = '2025-05-03 14:42:16.563049'
ayer = datetime.strptime(ayer, '%Y-%m-%d %H:%M:%S.%f')
#año
a1 = 2025
print(primero, ultimo, ayer)

20250228 20250228 2025-05-03 14:42:16.563049


**Se importa la base de datos de Fact_Garantias del periodo deseado (recordar que es mes vencido)**

In [6]:
cnxn = pyodbc.connect('DRIVER=SQL Server;Server=192.168.50.38\\DW_FZ;Database=DW_FZ;Port=1433;Trusted_Connection=yes')

Base de garantias 

In [7]:
query = f"""select * 
from (SELECT *
	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
     FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha = {ultimo}) a
  where rank_id=1;
"""

garantias_total = pd.read_sql_query(query, cnxn)
garantias_total.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_25144\2569993497.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  garantias_total = pd.read_sql_query(query, cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45724 entries, 0 to 45723
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_fecha                   45724 non-null  int64         
 1   credito                    45724 non-null  float64       
 2   codigo_sucursal            45724 non-null  int64         
 3   numero_id_deudor           45724 non-null  float64       
 4   nombre_deudor              45724 non-null  object        
 5   direccion_domicilio        45724 non-null  object        
 6   ciudad                     45724 non-null  object        
 7   telefono_domicilio         45724 non-null  object        
 8   telefono_empresa           45724 non-null  object        
 9   telefono_celular           45724 non-null  object        
 10  valor_desembolso           45724 non-null  float64       
 11  fecha_desembolso           45724 non-null  datetime64[ns]
 12  fech

In [8]:
garantias_total.tail()

,id_fecha,credito,codigo_sucursal,numero_id_deudor,nombre_deudor,direccion_domicilio,ciudad,telefono_domicilio,telefono_empresa,telefono_celular,valor_desembolso,fecha_desembolso,fecha_vencimiento,valor_saldo,dias_mora,valor_garantia,id_vehiculo,vehiculo_placa,vehiculo_marca,vehiculo_descripcion,vehiculo_color,vehiculo_modelo,vehiculo_codigo_fasecolda,vehiculo_cilindraje,vehiculo_numero_motor,vehiculo_numero_chasis,vehiculo_garantia_uso,codigo_garantia,concesionario,tipo_seguro_gps,gps_fecha_vencimiento,vlr_pri_seg_gps,numero_interno_poliza,numero_poliza,tipo_seguro_veh,ind_endo,poliza_fecha_inicio,poliza_fecha_vencimiento,valor_poliza,vlr_pri_seg_veh,memo,fecha_insercion,id_garantia,rank_id
45719,20250228,238668.0,114,901327959.0,GRUPO EMPRESARIAL RRL SAS,VEREDA SANTA BARBARA granja la caba¦a,ARBELAEZ CUNDIN,,000000000,0000000000,399002000.0,2024-09-19,2029-09-19,384336706.0,8,498000000.0,79111,NOK329,VOLQUETA Placa NOK329,VOLQUETA Marca CHEVROLET FVZ 700P FORWAR,BLANCO,2025,01626113,0,6HK1-291233,JALFVZ347S7000020,PUB,1,CENTRODIESEL,337,2029-09-19,68250.0,696628.0,,90,0,2024-09-19,2029-09-19,498000000.0,1166950.0,,2025-02-28 04:37:05.617,187268,1
45720,20250228,186543.0,614,901156430.0,CONSTRUMAQ DC SAS,CARRERA 24 No 17 - 51,SANTANDER BUCAR,,152601988,3152601988,250000000.0,2021-01-05,2026-01-05,76385922.0,22,239800000.0,31630,SZZ636,VOLQUETA Placa SZZ636,VOLQUETA INTERNATIONAL 7600 WORKSTAR ^35,BLANCO,2013,03626091,0,35300299,3HTWYAHT5DN197805,PUB,1,CASA TORO BARRANQUIL,0,1980-01-01,0.0,0.0,,0,0,1980-01-01,1980-01-01,0.0,0.0,,2025-02-28 04:37:05.617,143130,1
45721,20250228,156393.0,714,91001190.0,CHARRY MORENO HERNAN ELIACER,KR 40 3B 25,CESAR AGUACHICA,005662118,,3173425925,97800000.0,2018-10-16,2028-09-16,66548836.0,469,171500000.0,31688,TAW518,VOLQUETA Placa TAW518,VOLQUETA Marca FOTON AUMAN Modelo 2013 P,BLANCO,2013,18926001,0,1511J022507,LVBDLPJB2DH200003,PUB,1,FINANZAUTO,0,1980-01-01,0.0,0.0,,0,0,1980-01-01,1980-01-01,0.0,0.0,,2025-02-28 04:37:05.617,117801,1
45722,20250228,211786.0,714,80121226.0,PEDRAZA LOPEZ JOHN DANILO,CALLE 2A 17-27 TORRE B APTO 201,BOYACA SOGAMOSO,002111111,,3116216529,242100000.0,2022-06-30,2027-07-01,167153738.0,0,322100000.0,56363,TLP889,VOLQUETA Placa TLP889,VOLQUETA Marca HINO FM1A L7D ^500! MT 89,BLANCO,2018,03726015,0,A09CUP10132,9F3FM1AL7JXX10039,PUB,1,DIRECTO,337,2027-07-01,68250.0,531118.0,2000243165,292,1,2022-06-30,2023-06-30,322100000.0,0.0,,2025-02-28 04:37:05.617,164195,1
45723,20250228,217053.0,614,64587424.0,BALOCO BENITEZ YENIS YOJANA,CL 65 47 45 PI APTO 1,BARRANQUILLA AT,022218893,,3022218893,85000000.0,2022-12-12,2028-12-12,66687721.0,0,85000000.0,82111,NUL893,VOLSKWAGEN NUL893,VOLSKWAGEN TAOS COMNFORTLINE SELECT 2024,PLATA,2024,,0,DSJ109476,8AWBJ6B25RA812076,,1,,0,1980-01-01,0.0,730960.0,220037844-,30,0,2024-11-12,2028-12-12,366490000.0,1401254.0,,2025-02-28 04:37:05.617,190348,1


In [9]:
garantias_total = garantias_total[["credito", "vehiculo_codigo_fasecolda", "vehiculo_placa", "vehiculo_modelo", "valor_garantia"]]

In [10]:
garantias_total.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45724 entries, 0 to 45723
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   credito                    45724 non-null  float64
 1   vehiculo_codigo_fasecolda  45724 non-null  object 
 2   vehiculo_placa             45724 non-null  object 
 3   vehiculo_modelo            45724 non-null  int64  
 4   valor_garantia             45724 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB


Se importan los datos de la basa fasecolda

In [11]:
query = f"""
SELECT [Codigo], [Homologocodigo], [Marca], [Clase], [Referencia1], [Referencia2], [Referencia3], [Um],
[1970], [1971], [1972], [1973], [1974], [1975], [1976], [1977], [1978], [1979], [1980], [1981], [1982], 
[1983], [1984], [1985], [1986], [1987], [1988], [1989], [1990], [1991], [1992], [1993], [1994], [1995], 
[1996], [1997], [1998], [1999], [2000], [2001], [2002], [2003], [2004], [2005], [2006], [2007], [2008], 
[2009], [2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017], [2018], [2019], [2020], [2021], 
[2022], [2023], [2024], [2025], [2026] 
FROM [Analitica].[dbo].[Guia_fasecolda_nuevo_formato];
 ;
""" 
GuiaFasecolda = pd.read_sql_query(query,cnxn)

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_25144\2932517091.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  GuiaFasecolda = pd.read_sql_query(query,cnxn)


merge de guia_fasecolda con garatias 

In [12]:
# Realizamos el merge entre los DataFrames garantias_total y GuiaFasecolda
df_merged_garantias_guia = pd.merge(garantias_total, GuiaFasecolda, left_on="vehiculo_codigo_fasecolda", right_on="Codigo", how="left")


Se trae el historico de la Fact_Cartera_Mes, con el fin de obtener la garantia del primer comite 

In [13]:

query = """
WITH CTE AS (
    SELECT 
        credito, 
        valor_garantia,
        modelo_vehiculo,
        ROW_NUMBER() OVER (PARTITION BY credito ORDER BY id_fecha) AS row_num
    FROM Fact_Cartera_Mes
)
SELECT 
    credito, 
    valor_garantia,
    modelo_vehiculo
FROM CTE
WHERE row_num = 1
ORDER BY credito;
"""
hist_cartera = pd.read_sql(query, cnxn)
hist_cartera.info

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_25144\46807163.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hist_cartera = pd.read_sql(query, cnxn)


<bound method DataFrame.info of          credito  valor_garantia  modelo_vehiculo
0         7285.0      12814000.0             1986
1        10449.0             0.0             1993
2        13548.0      11000000.0             1994
3        13745.0             0.0             1991
4        17160.0      28695000.0             1993
...          ...             ...              ...
202514  246134.0     135200000.0             2023
202515  246135.0      29424871.0             2012
202516  246136.0       5000000.0             2022
202517  398036.0      75000000.0             2005
202518  935994.0      53500000.0             2005

[202519 rows x 3 columns]>

Merge de guia_fasecolda & garantias con cartera_mes

In [14]:
df_merged = pd.merge(df_merged_garantias_guia, hist_cartera, left_on= "credito", right_on ="credito", how="left")
#renombre de la columna para encontrar el valor del primer comite 
df_merged.rename(columns={
    
    'valor_garantia_y' :'valor_primer_comite'
    
}, inplace=True)

In [15]:
# # Convertir 'valor_primer_comite' a numérico
# df_merged["valor_primer_comite"] = pd.to_numeric(df_merged["valor_primer_comite"], errors='coerce')

# # Definir las columnas de años
# columns_years = [str(year) for year in range(1970, 2027)]

# # Crear una nueva columna 'valor_por_ano' para almacenar el valor de la columna correspondiente al año
# df_merged["valor_por_ano"] = None

# # Crear una columna para indicar el origen del valor
# df_merged["origen_valor"] = None

# # Crear una columna para indicar el origen del valor
# df_merged["valor_fasecolda"] = None


# # Iterar sobre las filas del DataFrame combinado
# for index, row in df_merged.iterrows():
#     # Obtener el año de la columna 'vehiculo_modelo'
#     year = str(row['vehiculo_modelo'])  # Convertir el año a string para la comparación
    
#     # Verificar si el año está en las columnas de los años
#     if year in columns_years:
#         # Asignar el valor de la columna correspondiente al año
#         df_merged.at[index, 'valor_por_ano'] = row[year]
#         df_merged.at[index, 'origen_valor'] = 'valor_fasecolda'
#         df_merged.at[index, 'valor_fasecolda'] = row[year]
         
#     elif row['valor_por_ano'] == 0 or row['valor_por_ano'] == '' or row['valor_por_ano'] is None:
#         # Si el valor es 0, vacío o None, reemplazarlo con el valor de la columna 'valor_primer_comite'
#         df_merged.at[index, 'valor_por_ano'] = row['valor_primer_comite']
#         df_merged.at[index, 'origen_valor'] = 'valor_primer_comite'
    
     
      
# # Aseguramos que la columna 'valor_por_ano' esté como numérica
# df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')
# df_merged['valor_fasecolda'] = pd.to_numeric(df_merged['valor_fasecolda'], errors='coerce')

# # Multiplicamos 'valor_por_ano' por 1000
# df_merged.loc[df_merged['origen_valor'] == 'valor_fasecolda', 'valor_por_ano'] *= 1000
# # Multiplicamos 'valor_por_ano' por 1000
# df_merged.loc[df_merged['origen_valor'] == 'valor_fasecolda', 'valor_fasecolda'] *= 1000
# # Mostrar el DataFrame actualizado
# df_merged.tail()


### Depreciación

In [16]:
# Convertir 'valor_primer_comite' a numérico
df_merged["valor_primer_comite"] = pd.to_numeric(df_merged["valor_primer_comite"], errors='coerce')

# Definir las columnas de años
columns_years = [str(year) for year in range(1970, 2027)]

# Crear una nueva columna 'valor_por_ano' para almacenar el valor de la columna correspondiente al año
df_merged["valor_por_ano"] = None

# Crear una columna para indicar el origen del valor
df_merged["origen_valor_fase"] = None

# Crear una columna para indicar el origen del valor
df_merged["valor_fasecolda"] = None


# Iterar sobre las filas del DataFrame combinado
for index, row in df_merged.iterrows():
    # Obtener el año de la columna 'vehiculo_modelo'
    year = str(row['vehiculo_modelo'])  # Convertir el año a string para la comparación
    
    # Verificar si el año está en las columnas de los años
    if year in columns_years:
        # Asignar el valor de la columna correspondiente al año
        df_merged.at[index, 'valor_por_ano'] = row[year]
        df_merged.at[index, 'origen_valor_fase'] = 'valor_fasecolda'
        df_merged.at[index, 'valor_fasecolda'] = row[year]

      
# Aseguramos que la columna 'valor_por_ano' esté como numérica
df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')
df_merged['valor_fasecolda'] = pd.to_numeric(df_merged['valor_fasecolda'], errors='coerce')

# Multiplicamos 'valor_por_ano' por 1000
df_merged.loc[df_merged['origen_valor_fase'] == 'valor_fasecolda', 'valor_por_ano'] *= 1000
# Multiplicamos 'valor_por_ano' por 1000
df_merged.loc[df_merged['origen_valor_fase'] == 'valor_fasecolda', 'valor_fasecolda'] *= 1000
# Mostrar el DataFrame actualizado
df_merged.tail()


,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_primer_comite,modelo_vehiculo,valor_por_ano,origen_valor_fase,valor_fasecolda
45719,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0,0.0,valor_fasecolda,0.0
45720,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0,239800000.0,valor_fasecolda,239800000.0
45721,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0,171500000.0,valor_fasecolda,171500000.0
45722,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400,322100000.0,2018.0,322100000.0,valor_fasecolda,322100000.0
45723,217053.0,,NUL893,2024,85000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129400000.0,2022.0,NaN,valor_fasecolda,NaN


In [17]:
# df_merged["origen_valor_comite"] = None

# # Recorrer nuevamente el DataFrame para realizar las correcciones
# for index, row in df_merged.iterrows():
#     # Verificar si 'valor_por_ano' es vacío, nulo o 0 y el origen es 'vehiculo_modelo'
#     if (pd.isna(row['valor_por_ano']) or row['valor_por_ano'] == 0 or str(row['valor_por_ano']).strip() == '') and row['origen_valor_fase'] == 'valor_fasecolda':
#         # Reemplazar 'valor_por_ano' con el valor de 'valor_primer_comite', verificando que no sea NaN
#         if not pd.isna(row['valor_primer_comite']):
#             df_merged.at[index, 'valor_por_ano'] = row['valor_primer_comite']
#             df_merged.at[index, 'origen_valor_comite'] = 'valor_primer_comite'
# # Asegurarnos de que la columna 'valor_por_ano' esté como numérica
# df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')


In [18]:
# # Se realiza for para validar el primer comite 
# ##Recorrer nuevamente el DataFrame para realizar las correcciones7

# # Iterar sobre las filas del DataFrame
# for index, row in df_merged.iterrows():
#     # Verificar si 'valor_por_ano' es vacío, nulo o 0 y el origen es 'valor_primer_comite'
#     if (pd.isna(row['valor_por_ano']) or row['valor_por_ano'] == 0 or str(row['valor_por_ano']).strip() == '') and row['origen_valor_comite'] == 'valor_primer_comite':
#         # Asignar vacío a la columna 'origen_valor' si la condición se cumple
#         df_merged.at[index, 'origen_valor_fase'] = ''
#         df_merged.at[index, 'origen_valor_comite'] = ''

# # Asegurarnos de que la columna 'valor_por_ano' esté como numérica
# df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')

# df_merged.tail()

In [19]:
#columna para calcular la depresiación 
# En caso de que vehivulo modelo sea igual a  0 o vacio se toma el valro de modelo_vehiculo que viene de fact_cartera_mes
a1 = 2025  # año actual

# Crear la nueva columna 'valor_calculado' con la fórmula y redondear a 2 decimales
df_merged["valor_calculado"] = df_merged.apply(
    lambda row: round(min(0.05 * (a1 - (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo'])), 1), 2) 
    if a1 > (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo']) 
    else 0,
    axis=1
)




In [20]:
#prueba 
df_merged["valor_depreciado_agrupado_comites_fase"] = df_merged.apply(
    lambda row: round(
        row['valor_primer_comite'] * (1 - row['valor_calculado']), 2
    ) if pd.isna(row['valor_por_ano'])  # Si valor_por_ano está vacío
    else round(
        row['valor_primer_comite'] * 0.75, 2  # Si valor_por_ano es 0, deprecia 25% de valor_garantia_y
    ) if row['valor_por_ano'] == 0  
    else round(
        row['valor_por_ano'] * 0.75, 2  # Si valor_por_ano tiene valor distinto de 0, deprecia 25% de valor_por_ano
    ), 
    axis=1
)


In [21]:
# #prueba  de valor garantias 
# df_merged["valor_depreciado_agrupado_comites_fase"] = df_merged.apply(
#     lambda row: round(
#         row['valor_por_ano'] * (1 - row['valor_calculado']), 2  # Depreciar según valor_por_ano y valor_calculado
#     ) if row['origen_valor_comite'] == 'valor_primer_comite'  # Si el origen es 'valor_primer_comite'
#     else round(
#         row['valor_por_ano'] * 0.75, 2  # Si no, depreciar un 25% de valor_por_ano
#     ),
#     axis=1
# )

# df_merged

In [22]:
df_merged["origen_valor_comite"] = None

# Recorrer nuevamente el DataFrame para realizar las correcciones
for index, row in df_merged.iterrows():
    # Verificar si 'valor_por_ano' es vacío, nulo o 0 y el origen es 'vehiculo_modelo'
    if (pd.isna(row['valor_por_ano']) or row['valor_por_ano'] == 0 or str(row['valor_por_ano']).strip() == '') and row['origen_valor_fase'] == 'valor_fasecolda' :
        df_merged.at[index, 'origen_valor_fase'] = ''
        # Reemplazar 'valor_por_ano' con el valor de 'valor_primer_comite', verificando que no sea NaN
        if not pd.isna(row['valor_primer_comite']):
            df_merged.at[index, 'valor_por_ano'] = row['valor_primer_comite']
            df_merged.at[index, 'origen_valor_comite'] = 'valor_primer_comite'
            
    elif (pd.isna(row['origen_valor_fase'])):
        df_merged.at[index, 'valor_por_ano'] = row['valor_primer_comite']
        df_merged.at[index, 'origen_valor_comite'] = 'valor_primer_comite'
        
# Asegurarnos de que la columna 'valor_por_ano' esté como numérica
df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')


In [23]:
# Se realiza for para validar el primer comite 
##Recorrer nuevamente el DataFrame para realizar las correcciones7

# Iterar sobre las filas del DataFrame
for index, row in df_merged.iterrows():
    # Verificar si 'valor_por_ano' es vacío, nulo o 0 y el origen es 'valor_primer_comite'
    if (pd.isna(row['valor_por_ano']) or row['valor_por_ano'] == 0 or str(row['valor_por_ano']).strip() == '') and row['origen_valor_comite'] == 'valor_primer_comite':
        # Asignar vacío a la columna 'origen_valor' si la condición se cumple
        df_merged.at[index, 'origen_valor_fase'] = ''
        df_merged.at[index, 'origen_valor_comite'] = ''

# Asegurarnos de que la columna 'valor_por_ano' esté como numérica
df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')

df_merged.tail()

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_primer_comite,modelo_vehiculo,valor_por_ano,origen_valor_fase,valor_fasecolda,valor_calculado,valor_depreciado_agrupado_comites_fase,origen_valor_comite
45719,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0,498000000.0,,0.0,0.00,373500000.0,valor_primer_comite
45720,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0,239800000.0,valor_fasecolda,239800000.0,0.60,179850000.0,None
45721,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0,171500000.0,valor_fasecolda,171500000.0,0.60,128625000.0,None
45722,211786.0,03726015,TLP889,2018,322100000.0,03726015,03765014,HINO,VOLQUETA,FM1A,L7D [500],MT 8900CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,322100,339100,356900,375700,403100,481400,0,0,481400,322100000.0,2018.0,322100000.0,valor_fasecolda,322100000.0,0.35,241575000.0,None
45723,217053.0,,NUL893,2024,85000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129400000.0,2022.0,129400000.0,,NaN,0.05,122930000.0,valor_primer_comite


## Carrocerias 

In [24]:
cyc = f"""Select t3.credito,
t3.id_fecha_desembolso,
LEFT(t3.id_fecha_desembolso, 6) AS anio
,t3.placa_vehiculo,
t4.precio_carroceria,
t4.precio_chasis,
t4.clasevehiculo,
t4.carroceria
from
(select t1.*,t2.numero_solicitud from
(select credito, placa_vehiculo, id_fecha_desembolso from dw_fz.dbo.Fact_Cartera_mes where id_fecha = 20250131 ) t1
left join (select credito, numero_solicitud from [DW_FZ].[dbo].[Fact_Cartera_Solicitud] group by credito,numero_solicitud)as t2 on t1.credito = t2.credito) t3
left join (select id_solicitud, precio_carroceria, precio_chasis, clasevehiculo, placa, carroceria
from [DW_FZ].[sol].[Dim_Sol_Vehiculos]) as t4 on t3.numero_solicitud = t4.id_solicitud and t3.placa_vehiculo = t4.placa
where t4.precio_carroceria >0 and ((t4.clasevehiculo != 'CAMIONETA PASAJ.' and (carroceria != 'VAN' or carroceria != 'FURGON')) or t4.clasevehiculo != 'AUTOMOVIL') order by t3.credito
"""
car_chas= pd.read_sql_query(cyc,cnxn)
car_chas.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_25144\2173901832.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_chas= pd.read_sql_query(cyc,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7840 entries, 0 to 7839
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   credito              7840 non-null   float64
 1   id_fecha_desembolso  7840 non-null   object 
 2   anio                 7840 non-null   object 
 3   placa_vehiculo       7840 non-null   object 
 4   precio_carroceria    7840 non-null   int64  
 5   precio_chasis        7840 non-null   int64  
 6   clasevehiculo        7840 non-null   object 
 7   carroceria           7840 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 490.1+ KB


In [25]:
car_chas["origen_valor"] = None

for index, row in car_chas.iterrows():
    # Verificar las condiciones y asignar el valor correspondiente a 'origen_valor'
    if row['precio_carroceria'] > 1:
        car_chas.at[index, 'origen_valor'] ="valor_carroceria"
car_chas

,credito,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,origen_valor
0,168303.0,20190628,201906,GFP841,33341000,140000000,CAMION,FURGON,valor_carroceria
1,168373.0,20190628,201906,GET582,14200000,55990000,CAMION,ESTACAS,valor_carroceria
2,168425.0,20190628,201906,GET645,35000000,46990000,CAMION,furgon,valor_carroceria
3,168539.0,20190628,201906,GET545,23000000,51990000,CAMION,ESTACAS,valor_carroceria
4,169205.0,20190722,201907,GET713,12500000,53899999,CAMION,ESTACAS,valor_carroceria
...,...,...,...,...,...,...,...,...,...
7835,244509.0,20250131,202501,NUV466,35911764,284900000,REMOLCADOR,SRS,valor_carroceria
7836,244513.0,20250131,202501,PMV235,16490000,86500000,CAMION,FURGON,valor_carroceria
7837,244514.0,20250131,202501,PMV432,25990000,90000000,CAMION,FURGON,valor_carroceria
7838,244515.0,20250131,202501,PMU991,17500000,69990000,CAMION,furgon,valor_carroceria


In [26]:
# Convertimos la columna 'id_fecha_desembolso' en formato 'YYYYMMDD' a formato datetime
car_chas['id_fecha_desembolso'] = pd.to_datetime(car_chas['id_fecha_desembolso'], format='%Y%m%d')

# Fecha de hoy
hoy = datetime.today()

# Calculamos la diferencia en años
car_chas['anios_cumplidos'] = (hoy - car_chas['id_fecha_desembolso']).dt.days // 365

In [27]:
#depreciación de las carrocerías 

# Calcular la columna 'tiempo_depre' con la lógica del vehículo
car_chas['tiempo_depre'] = np.where(car_chas['clasevehiculo'] == 'BUS / BUSETA / MICROBUS', 20, 
                                     np.where(car_chas['carroceria'] == 'TERMO', 7, 10))

# Calcular 'vl_cria_depre' de manera eficiente
car_chas['vl_cria_depre'] = car_chas['precio_carroceria'] / car_chas['tiempo_depre']

# Calcular 'depreciado_hoy' como la multiplicación de 'anios_cumplidos' y 'vl_cria_depre'
car_chas['depreciado_hoy'] = car_chas['anios_cumplidos'] * car_chas['vl_cria_depre']

# Usar np.where para asignar el valor final de la carrocería
car_chas['valor_final_carroceria'] = np.where(car_chas['depreciado_hoy'] == 0, 
                                              car_chas['precio_carroceria'], 
                                              car_chas['precio_carroceria'] - car_chas['depreciado_hoy'])

# Convertir 'valor_final_carroceria' a entero si es necesario
car_chas['valor_final_carroceria'] = car_chas['valor_final_carroceria'].astype(int)
car_chas = car_chas[car_chas['precio_carroceria'] != 1]

# Ver las primeras filas del DataFrame para comprobar
car_chas.head()

,credito,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,origen_valor,anios_cumplidos,tiempo_depre,vl_cria_depre,depreciado_hoy,valor_final_carroceria
0,168303.0,2019-06-28,201906,GFP841,33341000,140000000,CAMION,FURGON,valor_carroceria,5,10,3334100.0,16670500.0,16670500
1,168373.0,2019-06-28,201906,GET582,14200000,55990000,CAMION,ESTACAS,valor_carroceria,5,10,1420000.0,7100000.0,7100000
2,168425.0,2019-06-28,201906,GET645,35000000,46990000,CAMION,furgon,valor_carroceria,5,10,3500000.0,17500000.0,17500000
3,168539.0,2019-06-28,201906,GET545,23000000,51990000,CAMION,ESTACAS,valor_carroceria,5,10,2300000.0,11500000.0,11500000
4,169205.0,2019-07-22,201907,GET713,12500000,53899999,CAMION,ESTACAS,valor_carroceria,5,10,1250000.0,6250000.0,6250000


##### cruce de garantias con el valor de la carrocería

In [28]:
df_merged_garantias_carroceria = pd.merge(garantias_total, car_chas, on="credito", how="left")

df_merged_garantias_carroceria

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,origen_valor,anios_cumplidos,tiempo_depre,vl_cria_depre,depreciado_hoy,valor_final_carroceria
0,33810.0,,,0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67251.0,,,0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71972.0,,,0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88258.0,,,0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99004.0,,,0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45719,238668.0,01626113,NOK329,2025,498000000.0,2024-09-19,202409,NOK329,60000000.0,438000000.0,VOLQUETA,PLATON,valor_carroceria,0.0,10.0,6000000.0,0.0,60000000.0
45720,186543.0,03626091,SZZ636,2013,239800000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45721,156393.0,18926001,TAW518,2013,171500000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45722,211786.0,03726015,TLP889,2018,322100000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## hipotecarias

In [29]:
# Creacion de base con los creditos que tienen garantia hipotecaria

# Creacion de base con los creditos que tienen garantia hipotecaria

#Lectura de la base que contiene los creditos con garantia hipotecaria
hipotecario = pd.read_excel(r'\\192.168.40.9\garaje$\BackUps_Analitica\BACKUP IVAN\ORGANIGRAMA\LTV\LTV MENSUAL\Hipotecas\HIPOTECAS.xlsx',skiprows=1)
hipotecario = hipotecario[['CREDITO','VALOR AVALÚO','FECHA AVALÚO']] # se toman solo las columnas que resultan utiles para el proceso
hipotecario.info()

hipotecario[['CREDITO','VALOR AVALÚO']].value_counts() #Se verifica los valores que contiene respecto a las varianñes credito y el valor del avaluo

hipotecario['año_avaluo'] = pd.to_datetime(hipotecario['FECHA AVALÚO'],format='mixed')
hipotecario['año_avaluo'] = [hipotecario['año_avaluo'][i].year for i in hipotecario.index]
hipotecario['año_avaluo'] = hipotecario['año_avaluo'].replace(1970,0)

print(hipotecario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 564.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
 3   año_avaluo    18 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 708.0 bytes
None


In [30]:
#for para realizar la visualización de las garantias hipotecarias para despues cruzarlas con la base 

hipotecario ["origen_valor"] = None

for index, row in hipotecario.iterrows():
    hipotecario ["origen_valor"] = "valor_hipoteca"
    
hipotecario

,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,origen_valor
0,214219,76000000,2016-11-09,2016,valor_hipoteca
1,156806,3561000000,2018-10-03,2018,valor_hipoteca
2,178584,112672000,2018-12-17,2018,valor_hipoteca
3,169184,112800000,2019-05-31,2019,valor_hipoteca
4,180396,187845000,2019-11-02,2019,valor_hipoteca
5,183214,110382000,2020-08-06,2020,valor_hipoteca
6,187180,1389765000,2020-11-12,2020,valor_hipoteca
7,188717,148816821,2020-12-24,2020,valor_hipoteca
8,217808,883284000,2019-01-08,2019,valor_hipoteca
9,194613,1037680000,2019-01-08,2019,valor_hipoteca


In [31]:
año_actual = ayer.year


vlr_apreciacion = np.where(hipotecario['año_avaluo']!=0,(((año_actual - hipotecario['año_avaluo'])*0.04)+1),0)
apreciacion = hipotecario['VALOR AVALÚO'] * vlr_apreciacion

#A la data Hipotecario se adiciona la columna con el valor de apreciacion de cada inmueble
hipotecario['vlr_avaluo_apreciado'] = apreciacion.round(2)
hipotecario ['CREDITO'] = hipotecario['CREDITO'].astype(float)
print(hipotecario.info())
hipotecario

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CREDITO               18 non-null     float64       
 1   VALOR AVALÚO          18 non-null     int64         
 2   FECHA AVALÚO          18 non-null     datetime64[ns]
 3   año_avaluo            18 non-null     int64         
 4   origen_valor          18 non-null     object        
 5   vlr_avaluo_apreciado  18 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 996.0+ bytes
None


,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,origen_valor,vlr_avaluo_apreciado
0,214219.0,76000000,2016-11-09,2016,valor_hipoteca,1.033600e+08
1,156806.0,3561000000,2018-10-03,2018,valor_hipoteca,4.558080e+09
2,178584.0,112672000,2018-12-17,2018,valor_hipoteca,1.442202e+08
3,169184.0,112800000,2019-05-31,2019,valor_hipoteca,1.398720e+08
4,180396.0,187845000,2019-11-02,2019,valor_hipoteca,2.329278e+08
5,183214.0,110382000,2020-08-06,2020,valor_hipoteca,1.324584e+08
6,187180.0,1389765000,2020-11-12,2020,valor_hipoteca,1.667718e+09
7,188717.0,148816821,2020-12-24,2020,valor_hipoteca,1.785802e+08
8,217808.0,883284000,2019-01-08,2019,valor_hipoteca,1.095272e+09
9,194613.0,1037680000,2019-01-08,2019,valor_hipoteca,1.286723e+09


In [32]:
df_merged_garantias_hipotecas = pd.merge(garantias_total, hipotecario, left_on="credito", right_on="CREDITO", how="left")
df_merged_garantias_hipotecas

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,origen_valor,vlr_avaluo_apreciado
0,33810.0,,,0,0.0,NaN,NaN,NaT,NaN,NaN,NaN
1,67251.0,,,0,0.0,NaN,NaN,NaT,NaN,NaN,NaN
2,71972.0,,,0,0.0,NaN,NaN,NaT,NaN,NaN,NaN
3,88258.0,,,0,0.0,NaN,NaN,NaT,NaN,NaN,NaN
4,99004.0,,,0,0.0,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
45719,238668.0,01626113,NOK329,2025,498000000.0,NaN,NaN,NaT,NaN,NaN,NaN
45720,186543.0,03626091,SZZ636,2013,239800000.0,NaN,NaN,NaT,NaN,NaN,NaN
45721,156393.0,18926001,TAW518,2013,171500000.0,NaN,NaN,NaT,NaN,NaN,NaN
45722,211786.0,03726015,TLP889,2018,322100000.0,NaN,NaN,NaT,NaN,NaN,NaN


In [33]:
df_combined = pd.merge(df_merged,  hipotecario, left_on="credito", right_on="CREDITO", how="left")
df_combined     

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_primer_comite,modelo_vehiculo,valor_por_ano,origen_valor_fase,valor_fasecolda,valor_calculado,valor_depreciado_agrupado_comites_fase,origen_valor_comite,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,origen_valor,vlr_avaluo_apreciado
0,33810.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12500000.0,1994.0,12500000.0,None,NaN,1.00,0.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
1,67251.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28990000.0,2011.0,28990000.0,None,NaN,0.70,8697000.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
2,71972.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49990000.0,2011.0,49990000.0,None,NaN,0.70,14997000.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
3,88258.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28490000.0,2013.0,28490000.0,None,NaN,0.60,11396000.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
4,99004.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35290000.0,2014.0,35290000.0,None,NaN,0.55,15880500.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45719,238668.0,01626113,NOK329,2025,498000000.0,01626113,01665044,CHEVROLET,VOLQUETA,FVZ,700P FORWARD [CORTO] Euro IV,MT 7800CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,243500,253600,264200,275200,286700,298600,320400,384600,0,0,500400,498000000.0,2025.0,498000000.0,,0.0,0.00,373500000.0,valor_primer_comite,NaN,NaN,NaT,NaN,NaN,NaN
45720,186543.0,03626091,SZZ636,2013,239800000.0,03626091,03665017,INTERNATIONAL,VOLQUETA,7600,WORKSTAR [350HP],MT TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,187200,199200,211900,225400,239800,255100,271400,0,0,0,0,0,0,0,0,0,0,323100,334390000.0,2020.0,239800000.0,valor_fasecolda,239800000.0,0.60,179850000.0,None,NaN,NaN,NaT,NaN,NaN,NaN
45721,156393.0,18926001,TAW518,2013,171500000.0,18926001,18965001,FOTON,VOLQUETA,AUMAN,BJ3253DLPJB-7,MT 9700CC TD 6X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147100,154800,162900,171500,180500,190000,0,0,0,0,0,0,0,0,0,0,275000,122800000.0,2013.0,171500000.0,valor_fasecolda,171500000.0,0.60,1

### Combinación de los merged 

In [34]:
# Combinar el primer DataFrame (df_merged) con el segundo (df_merged_garantias_carroceria) usando 'credito' como clave

 
df_combined = pd.merge(df_merged, df_merged_garantias_carroceria, on='credito', how='left')

df_combined.columns


Index(['credito', 'vehiculo_codigo_fasecolda_x', 'vehiculo_placa_x',
       'vehiculo_modelo_x', 'valor_garantia_x', 'Codigo', 'Homologocodigo',
       'Marca', 'Clase', 'Referencia1', 'Referencia2', 'Referencia3', 'Um',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
       '2024', '2025', '2026', 'valor_primer_comite', 'modelo_vehiculo',
       'valor_por_ano', 'origen_valor_fase', 'valor_fasecolda',
       'valor_calculado', 'valor_depreciado_agrupado_comites_fase',
       'origen_valor_comite', 'vehiculo_codigo_fasecolda_y',
       'vehiculo_placa_y', 'vehiculo_mo

In [35]:
# Seleccionar solo las columnas necesarias
df_combined = df_combined[['credito','vehiculo_placa_x', 'clasevehiculo',  'valor_primer_comite', 'valor_fasecolda', 'valor_por_ano', 'valor_depreciado_agrupado_comites_fase', 'precio_carroceria', 'precio_chasis', 'valor_final_carroceria', 'origen_valor' ,'origen_valor_fase', 'origen_valor_comite']]
df_combined

,credito,vehiculo_placa_x,clasevehiculo,valor_primer_comite,valor_fasecolda,valor_por_ano,valor_depreciado_agrupado_comites_fase,precio_carroceria,precio_chasis,valor_final_carroceria,origen_valor,origen_valor_fase,origen_valor_comite
0,33810.0,,NaN,12500000.0,NaN,12500000.0,0.0,NaN,NaN,NaN,NaN,None,valor_primer_comite
1,67251.0,,NaN,28990000.0,NaN,28990000.0,8697000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite
2,71972.0,,NaN,49990000.0,NaN,49990000.0,14997000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite
3,88258.0,,NaN,28490000.0,NaN,28490000.0,11396000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite
4,99004.0,,NaN,35290000.0,NaN,35290000.0,15880500.0,NaN,NaN,NaN,NaN,None,valor_primer_comite
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52879,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None
52880,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None
52881,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None
52882,211786.0,TLP889,NaN,322100000.0,322100000.0,322100000.0,241575000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None


###

In [36]:
df_final = pd.merge(df_combined, df_merged_garantias_hipotecas, left_on='credito', right_on="CREDITO", how='left')
df_final

,credito_x,vehiculo_placa_x,clasevehiculo,valor_primer_comite,valor_fasecolda,valor_por_ano,valor_depreciado_agrupado_comites_fase,precio_carroceria,precio_chasis,valor_final_carroceria,origen_valor_x,origen_valor_fase,origen_valor_comite,credito_y,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,origen_valor_y,vlr_avaluo_apreciado
0,33810.0,,NaN,12500000.0,NaN,12500000.0,0.0,NaN,NaN,NaN,NaN,None,valor_primer_comite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,67251.0,,NaN,28990000.0,NaN,28990000.0,8697000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,71972.0,,NaN,49990000.0,NaN,49990000.0,14997000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,88258.0,,NaN,28490000.0,NaN,28490000.0,11396000.0,NaN,NaN,NaN,NaN,None,valor_primer_comite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,99004.0,,NaN,35290000.0,NaN,35290000.0,15880500.0,NaN,NaN,NaN,NaN,None,valor_primer_comite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52887,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
52888,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
52889,156393.0,TAW518,NaN,122800000.0,171500000.0,171500000.0,128625000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
52890,211786.0,TLP889,NaN,322100000.0,322100000.0,322100000.0,241575000.0,NaN,NaN,NaN,NaN,valor_fasecolda,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [37]:
df_final = df_final[['credito_x','vehiculo_placa_x',  'valor_primer_comite', 'valor_fasecolda', 'valor_por_ano', 'precio_carroceria', 'valor_depreciado_agrupado_comites_fase', 'vlr_avaluo_apreciado', 'VALOR AVALÚO', 'valor_final_carroceria',  'origen_valor_y' ,'origen_valor_x' ,'origen_valor_fase', 'origen_valor_comite']]

df_final

,credito_x,vehiculo_placa_x,valor_primer_comite,valor_fasecolda,valor_por_ano,precio_carroceria,valor_depreciado_agrupado_comites_fase,vlr_avaluo_apreciado,VALOR AVALÚO,valor_final_carroceria,origen_valor_y,origen_valor_x,origen_valor_fase,origen_valor_comite
0,33810.0,,12500000.0,NaN,12500000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
1,67251.0,,28990000.0,NaN,28990000.0,NaN,8697000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
2,71972.0,,49990000.0,NaN,49990000.0,NaN,14997000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
3,88258.0,,28490000.0,NaN,28490000.0,NaN,11396000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
4,99004.0,,35290000.0,NaN,35290000.0,NaN,15880500.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52887,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
52888,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
52889,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
52890,211786.0,TLP889,322100000.0,322100000.0,322100000.0,NaN,241575000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None


In [38]:
# Eliminar duplicados basados en las columnas 'Vehiculo_placa_x' y 'credito'
df_final_unique = df_final.drop_duplicates(subset=['vehiculo_placa_x', 'credito_x'])
df_final_unique


,credito_x,vehiculo_placa_x,valor_primer_comite,valor_fasecolda,valor_por_ano,precio_carroceria,valor_depreciado_agrupado_comites_fase,vlr_avaluo_apreciado,VALOR AVALÚO,valor_final_carroceria,origen_valor_y,origen_valor_x,origen_valor_fase,origen_valor_comite
0,33810.0,,12500000.0,NaN,12500000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
1,67251.0,,28990000.0,NaN,28990000.0,NaN,8697000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
2,71972.0,,49990000.0,NaN,49990000.0,NaN,14997000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
3,88258.0,,28490000.0,NaN,28490000.0,NaN,11396000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
4,99004.0,,35290000.0,NaN,35290000.0,NaN,15880500.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52884,238668.0,NOK329,498000000.0,0.0,498000000.0,60000000.0,373500000.0,NaN,NaN,60000000.0,NaN,valor_carroceria,,valor_primer_comite
52885,186543.0,SZZ636,334390000.0,239800000.0,239800000.0,NaN,179850000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
52887,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
52890,211786.0,TLP889,322100000.0,322100000.0,322100000.0,NaN,241575000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None


In [39]:
df_final_unique = df_final_unique.reset_index(drop=True)

df_final_unique

,credito_x,vehiculo_placa_x,valor_primer_comite,valor_fasecolda,valor_por_ano,precio_carroceria,valor_depreciado_agrupado_comites_fase,vlr_avaluo_apreciado,VALOR AVALÚO,valor_final_carroceria,origen_valor_y,origen_valor_x,origen_valor_fase,origen_valor_comite
0,33810.0,,12500000.0,NaN,12500000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
1,67251.0,,28990000.0,NaN,28990000.0,NaN,8697000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
2,71972.0,,49990000.0,NaN,49990000.0,NaN,14997000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
3,88258.0,,28490000.0,NaN,28490000.0,NaN,11396000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
4,99004.0,,35290000.0,NaN,35290000.0,NaN,15880500.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45719,238668.0,NOK329,498000000.0,0.0,498000000.0,60000000.0,373500000.0,NaN,NaN,60000000.0,NaN,valor_carroceria,,valor_primer_comite
45720,186543.0,SZZ636,334390000.0,239800000.0,239800000.0,NaN,179850000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
45721,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
45722,211786.0,TLP889,322100000.0,322100000.0,322100000.0,NaN,241575000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None


## traer_fact_cartera_mes con el fin de cruzar

In [40]:
query = f"""
SELECT id_fecha, credito, id_fecha_desembolso, tipo_id_deudor, numero_id_deudor, id_vehiculo, placa_vehiculo, 
    marca_vehiculo, color_vehiculo, licencia_vehiculo, linea_vehiculo,tasa_inicial, tasa_actual, tasa_nominal,
    modelo_vehiculo, plazo_total, valor_desembolso, numero_dias_mora, codigo_oficial, 
    estado_cartera, valor_garantia, saldo_capital, total_deuda, tipo_producto 
FROM DW_FZ.dbo.Fact_Cartera_Mes
WHERE (id_fecha = N'{ultimo}') AND tipo_producto IN ('10','30','35','37','70') ;
""" 
cartera_mes = pd.read_sql_query(query,cnxn)
cartera_mes.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_25144\208999830.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cartera_mes = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42780 entries, 0 to 42779
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_fecha             42780 non-null  object 
 1   credito              42780 non-null  float64
 2   id_fecha_desembolso  42780 non-null  object 
 3   tipo_id_deudor       42780 non-null  int64  
 4   numero_id_deudor     42780 non-null  float64
 5   id_vehiculo          42780 non-null  int64  
 6   placa_vehiculo       42780 non-null  object 
 7   marca_vehiculo       42780 non-null  object 
 8   color_vehiculo       42780 non-null  object 
 9   licencia_vehiculo    42780 non-null  object 
 10  linea_vehiculo       42780 non-null  object 
 11  tasa_inicial         42780 non-null  float64
 12  tasa_actual          42780 non-null  float64
 13  tasa_nominal         42780 non-null  float64
 14  modelo_vehiculo      42780 non-null  int64  
 15  plazo_total          42780 non-null 

### merge de con fact_cartera_mes 

In [41]:
df_unique_cartera = pd.merge(cartera_mes, df_final_unique, right_on='credito_x', left_on="credito", how='right')

# Usamos .fillna() para reemplazar valores nulos o vacíos
df_unique_cartera['id_fecha'] = df_unique_cartera['id_fecha'].fillna(ultimo)

df_unique_cartera


,id_fecha,credito,id_fecha_desembolso,tipo_id_deudor,numero_id_deudor,id_vehiculo,placa_vehiculo,marca_vehiculo,color_vehiculo,licencia_vehiculo,linea_vehiculo,tasa_inicial,tasa_actual,tasa_nominal,modelo_vehiculo,plazo_total,valor_desembolso,numero_dias_mora,codigo_oficial,estado_cartera,valor_garantia,saldo_capital,total_deuda,tipo_producto,credito_x,vehiculo_placa_x,valor_primer_comite,valor_fasecolda,valor_por_ano,precio_carroceria,valor_depreciado_agrupado_comites_fase,vlr_avaluo_apreciado,VALOR AVALÚO,valor_final_carroceria,origen_valor_y,origen_valor_x,origen_valor_fase,origen_valor_comite
0,20250228,33810.0,20040216,1.0,1.142881e+07,39.0,BER098,CHEVROLET,AZUL COBALTO,PAR,SWIFT 1300 CC,29.4999,26.2999,1.96,1994.0,48.0,10408237.0,999.0,111,R,8000000.0,2.754805e+06,4.315381e+06,70.0,33810.0,,12500000.0,NaN,12500000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
1,20250228,67251.0,20101013,1.0,1.941904e+07,30327.0,RDS163,CHEVROLET,AZUL NORUEGA,PAR,AVEO 1598C.C. M/T,20.9830,20.9830,1.59,2011.0,60.0,20200000.0,999.0,120,R,18400000.0,4.822491e+06,5.052330e+06,70.0,67251.0,,28990000.0,NaN,28990000.0,NaN,8697000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
2,20250228,71972.0,20110504,1.0,4.145923e+07,1.0,RIW109,CHEVROLET,NEGRO CARBONO,PAR,CRUZE M/T 1796C.C.,18.1559,18.1559,1.39,2011.0,60.0,39990000.0,999.0,120,P,14300000.0,1.083969e+06,2.668900e+06,70.0,71972.0,,49990000.0,NaN,49990000.0,NaN,14997000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
3,20250228,88258.0,20130219,1.0,7.991168e+07,1.0,RDY173,NISSAN,AZUL OSCURO,PAR,D22/NP300 . .,15.3894,15.3894,1.19,2011.0,61.0,25641000.0,999.0,127,P,13200000.0,1.535195e+06,1.689041e+06,70.0,88258.0,,28490000.0,NaN,28490000.0,NaN,11396000.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
4,20250228,99004.0,20140416,1.0,1.030626e+09,34378.0,WGO406,CHEVROLET,BLANCO LUNA,PUB,N300. 1206 CC,26.5260,26.5260,1.97,2014.0,49.0,25290000.0,999.0,127,J,32300000.0,1.420370e+06,1.420370e+06,70.0,99004.0,,35290000.0,NaN,35290000.0,NaN,15880500.0,NaN,NaN,NaN,NaN,NaN,None,valor_primer_comite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45719,20250228,238668.0,20240919,2.0,9.013280e+08,79111.0,NOK329,CHEVROLET,BLANCO NIEBLA,PUB,FVZ 700P FORWARD ^CO,27.5722,26.2999,1.96,2025.0,60.0,399002000.0,9.0,138,,498000000.0,3.843367e+08,3.866049e+08,10.0,238668.0,NOK329,498000000.0,0.0,498000000.0,60000000.0,373500000.0,NaN,NaN,60000000.0,NaN,valor_carroceria,,valor_primer_comite
45720,20250228,186543.0,20210105,2.0,9.011564e+08,1.0,MC15530,JOHN DEERE,AMARILLO,PAR,310L JHON DEERE,26.5260,26.2999,1.96,2020.0,60.0,250000000.0,23.0,257,P,334390000.0,7.638592e+07,7.674882e+07,10.0,186543.0,SZZ636,334390000.0,239800000.0,239800000.0,NaN,179850000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
45721,20250228,156393.0,20181016,1.0,9.100119e+07,31688.0,TAW518,FOTON,BLANCO .,PUB,AUMAN BJ3253DLPJB-7,29.6882,26.2999,1.96,2013.0,119.0,97800000.0,470.0,343,J,171500000.0,6.654884e+07,6.654884e+07,70.0,156393.0,TAW518,122800000.0,171500000.0,171500000.0,NaN,128625000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None
45722,20250228,211786.0,20220630,1.0,8.012123e+07,56363.0,TLP889,HINO,BLANCO VERDE,PUB,FM1A L7D ^500! MT 89,26.8241,26.2999,1.96,2018.0,60.0,242100000.0,0.0,493,J,322100000.0,1.671537e+08,1.671957e+08,10.0,211786.0,TLP889,322100000.0,322100000.0,322100000.0,NaN,241575000.0,NaN,NaN,NaN,NaN,NaN,valor_fasecolda,None


In [43]:
df_final_garantias = df_unique_cartera.reindex(columns=['id_fecha', 'credito_x', 
        'vehiculo_placa_x', 'valor_primer_comite', 'valor_fasecolda',  'precio_carroceria', 'VALOR AVALÚO', 'valor_final_carroceria',
       'valor_depreciado_agrupado_comites_fase','vlr_avaluo_apreciado', 'origen_valor_fase', 'origen_valor_comite', 'origen_valor_x', 'origen_valor_y'])

df_final_garantias['valor_garantia_final'] = (
    df_final_garantias['valor_depreciado_agrupado_comites_fase'].fillna(0) +
    df_final_garantias['valor_final_carroceria'].fillna(0) + 
    df_final_garantias['vlr_avaluo_apreciado'].fillna(0)
)
df_final_garantias.rename(columns={
    'vehiculo_placa_x': 'placa_vehiculo',
    'VALOR AVALÚO': 'valor_hipoteca',
    'origen_valor_x': 'origen_carroceria', 
    'origen_valor_y': 'origen_hipoteca',
    'credito_x': 'credito'
}, inplace=True)


df_final_garantias

,id_fecha,credito,placa_vehiculo,valor_primer_comite,valor_fasecolda,precio_carroceria,valor_hipoteca,valor_final_carroceria,valor_depreciado_agrupado_comites_fase,vlr_avaluo_apreciado,origen_valor_fase,origen_valor_comite,origen_carroceria,origen_hipoteca,valor_garantia_final
0,20250228,33810.0,,12500000.0,NaN,NaN,NaN,NaN,0.0,NaN,None,valor_primer_comite,NaN,NaN,0.0
1,20250228,67251.0,,28990000.0,NaN,NaN,NaN,NaN,8697000.0,NaN,None,valor_primer_comite,NaN,NaN,8697000.0
2,20250228,71972.0,,49990000.0,NaN,NaN,NaN,NaN,14997000.0,NaN,None,valor_primer_comite,NaN,NaN,14997000.0
3,20250228,88258.0,,28490000.0,NaN,NaN,NaN,NaN,11396000.0,NaN,None,valor_primer_comite,NaN,NaN,11396000.0
4,20250228,99004.0,,35290000.0,NaN,NaN,NaN,NaN,15880500.0,NaN,None,valor_primer_comite,NaN,NaN,15880500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45719,20250228,238668.0,NOK329,498000000.0,0.0,60000000.0,NaN,60000000.0,373500000.0,NaN,,valor_primer_comite,valor_carroceria,NaN,433500000.0
45720,20250228,186543.0,SZZ636,334390000.0,239800000.0,NaN,NaN,NaN,179850000.0,NaN,valor_fasecolda,None,NaN,NaN,179850000.0
45721,20250228,156393.0,TAW518,122800000.0,171500000.0,NaN,NaN,NaN,128625000.0,NaN,valor_fasecolda,None,NaN,NaN,128625000.0
45722,20250228,211786.0,TLP889,322100000.0,322100000.0,NaN,NaN,NaN,241575000.0,NaN,valor_fasecolda,None,NaN,NaN,241575000.0


In [45]:
##cierre de conexion
cnxn.close

<function Connection.close>

### Acá voy

In [44]:
df_final_garantias.to_excel(f"C:/Users/cristian.parra/Documents/DataPro/discriminadoGarantias{ultimo}.xlsx", index=False)

